# Step2b: OHE and SS with respect to X_train

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [14]:
X_train = pd.read_csv('../data/X_train.csv', index_col=0)

In [15]:
X_train

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,...,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,phone_prefix,total_minutes,total_calls,total_charge
1263,MA,40,408,no,yes,31,224.7,69,38.20,134.5,...,104,5.41,7.5,5,2.03,1,351,487.0,259,57.07
5,AL,118,510,yes,no,0,223.4,98,37.98,220.6,...,118,9.18,6.3,6,1.70,0,391,654.2,323,67.61
2214,CT,90,415,no,no,0,175.9,111,29.90,285.2,...,122,6.79,13.0,7,3.51,1,347,624.9,355,64.44
130,NE,90,415,no,no,0,145.5,92,24.74,217.7,...,123,6.61,10.9,2,2.94,3,353,521.0,331,52.79
3196,OR,124,510,no,no,0,169.3,108,28.78,178.6,...,82,10.90,12.2,3,3.29,1,337,602.4,284,58.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,MO,86,415,no,no,0,83.5,96,14.20,221.1,...,75,15.74,12.6,3,3.40,0,385,666.9,237,52.13
417,AZ,117,408,no,no,0,239.9,84,40.78,174.8,...,93,9.43,9.8,2,2.65,0,417,634.0,285,67.72
1458,MD,125,408,no,no,0,137.1,94,23.31,209.8,...,114,10.73,8.6,4,2.32,1,349,593.9,295,54.19
1004,DE,64,415,no,yes,27,201.3,101,34.22,143.8,...,127,6.76,12.3,3,3.32,1,402,507.6,320,56.52


In [16]:
X_train = X_train.drop(columns = ['state', 'phone_prefix'])

In [17]:
X_train.shape


(2333, 21)

In [18]:
X_train['area_code'] = X_train['area_code'].astype(str)

### Define categorical and numeric columns

In [19]:
categorical = []
numeric = []
for c in X_train.columns:
    if X_train[c].dtypes == 'object':
        categorical.append(c)
    else:
        numeric.append(c)
        
categorical, numeric

(['area_code', 'international_plan', 'voice_mail_plan'],
 ['account_length',
  'number_vmail_messages',
  'total_day_minutes',
  'total_day_calls',
  'total_day_charge',
  'total_eve_minutes',
  'total_eve_calls',
  'total_eve_charge',
  'total_night_minutes',
  'total_night_calls',
  'total_night_charge',
  'total_intl_minutes',
  'total_intl_calls',
  'total_intl_charge',
  'customer_service_calls',
  'total_minutes',
  'total_calls',
  'total_charge'])

### OHE

In [20]:
X_ohe = X_train.drop(columns = numeric)

In [21]:
ohe = OneHotEncoder(drop='first', sparse=False)

In [22]:
ohe.fit_transform(X_ohe)

array([[0., 0., 0., 1.],
       [0., 1., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [1., 0., 0., 1.],
       [0., 1., 1., 0.]])

In [23]:
categorical

['area_code', 'international_plan', 'voice_mail_plan']

In [24]:
X_ohe_df = pd.DataFrame(ohe.fit_transform(X_ohe), columns = ohe.get_feature_names())

### StandardScaler

In [25]:
X_ss = X_train.drop(columns = categorical)

In [26]:
X_ss.columns

Index(['account_length', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge', 'customer_service_calls',
       'total_minutes', 'total_calls', 'total_charge'],
      dtype='object')

In [27]:
ss = StandardScaler()
X_ss_df = pd.DataFrame(ss.fit_transform(X_ss), columns = X_ss.columns)

In [28]:
X_ss_df;

In [29]:
X_train_trans = pd.concat([X_ss_df, X_ohe_df], axis = 1)

### Export data

In [30]:
### X_train (original), X_train_trans (transformed), X_test, y_train, y_test
X_train_trans.to_csv('../data/X_train_trans2.csv')